In [ ]:
# the four packages below are the most useful and basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
# a couple of more advanced packages
from scipy.stats.stats import pearsonr,spearmanr
from matplotlib_venn import venn2
import mygene  

mg = mygene.MyGeneInfo()
#very important line that enables showing plots within jupyter notebook
%matplotlib inline

## select enhancer

In [ ]:
%%bash
export PATH=$PATH:/bioinformatics/homer/bin/

peak_file='../fetal_peak_atac/Fetal_peaks_164.txt'
output='../data/fetal_peak_anno.txt'

annotatePeaks.pl $peak_file hg38 -norm 1e7 > $output
echo "done all"

In [ ]:
annot_file = '../data/fetal_peak_anno.txt'
annot_file_df = pd.read_csv(annot_file, sep='\t', index_col=0)
annots = annot_file_df['Annotation']

filtered_df1 = annot_file_df.loc[annots=='Intergenic'] 
filtered_df2 = annot_file_df.loc[annot_file_df.Annotation.str.contains('intron')]
filtered_df=pd.concat([filtered_df1,filtered_df2])
scaled_size = 500
mid_points = (filtered_df.loc[:,'Start']+filtered_df.loc[:,'End'])//2
filtered_df.loc[:,'Start'] = mid_points - int(scaled_size/2)
filtered_df.loc[:,'End'] = mid_points + int(scaled_size/2)
filtered_df.iloc[:,:6].to_csv(annot_file.replace('.txt', '_scaled.txt'), sep='\t')

In [ ]:
%%bash
export PATH=$PATH:/bioinformatics/homer/bin/
peak_file=../data/fetal_peak_anno_scaled.txt
annot_file=../data/h3k27ac_fetal_peak_anno_scaled.txt
tag_dir='../fetal_chip_tag/chip/combined_h3k27ac'
annotatePeaks.pl $peak_file hg38 -norm 1e7 -d $tag_dir > $annot_file

In [ ]:
directory = './'
annot_file = '../data/h3k27ac_fetal_peak_anno_scaled.txt'
annot_df = pd.read_csv(directory + annot_file, sep='\t', index_col=0)
annot_df.index.name = None
annot = np.array(annot_df.loc[:,'Annotation'])
select_region=annot_df.loc[(annot_df.iloc[:,18]>16) ]
scaled_size = 200
mid_points = (select_region.loc[:,'Start']+select_region.loc[:,'End'])//2
select_region.loc[:,'Start'] = mid_points - int(scaled_size/2)
select_region.loc[:,'End'] = mid_points + int(scaled_size/2)
select_region.to_csv("../data/enhancer_fetal_cutoff16tag.txt",sep = "\t")

## search TF motifs from enhancer

In [ ]:
%%bash 
export PATH=$PATH:/bioinformatics/homer/bin/
#mkdir -p motif
motif_lib=~/daima/python/merge_motif_Jun2020/jaspar/Homer_format_motif_JASPAR.txt #http://jaspar.genereg.net/matrix-clusters/vertebrates/?detail=true

outfile=../motif/fetal_enhancer_motif.txt
findMotifsGenome.pl ../data/enhancer_fetal_cutoff16tag.txt hg38 ../motif/ -size 200 -p 40 -find $motif_lib > $outfile

## build TF-gene network

In [ ]:
%%bash
export PATH=$PATH:/bioinformatics/homer/bin/

mergePeaks -d 100 ../data/enhancer_fetal_cutoff16tag.txt ../data/enhancer_fetal_cutoff16tag.txt > ../data/fetal_adult_merged_peak.txt

In [ ]:
merged_peak = pd.read_csv('../data/fetal_adult_merged_peak.txt',sep='\t',header=0,index_col=0)
fetal_unique_enhancer= merged_peak[merged_peak["../data/enhancer_fetal_cutoff16tag.txt"].notnull() & 
                                merged_peak["../data/enhancer_adult_cutoff16tag.txt"].isnull() ]
fetal_unique_enhancer_id =list(fetal_unique_enhancer["../data/enhancer_fetal_cutoff16tag.txt"])


In [ ]:
fetal_enhancer = pd.read_csv('../data/enhancer_fetal_cutoff16tag.txt',sep='\t',header=0,index_col=0)
fetal_motif = pd.read_csv("../motif/fetal_enhancer_motif.txt",sep='\t',index_col=0)
fetal_motif["Motif Name New"]=fetal_motif.apply(lambda x: x["Motif Name"].split("$")[0],axis=1)
#geneout=mg.querymany(enhancer["Nearest PromoterID"], scopes='refseq',species="human",as_dataframe=True,fields='symbol')
#enhancer['geneSymbol']=list(geneout['symbol'])
#enhancer=enhancer[enhancer['geneSymbol'].isna()==False]  # only keep ones with gene symbols
fetal_enhancer=fetal_enhancer[abs(fetal_enhancer["Distance to TSS"])<500000] # limit distance
fetal_enhancer=fetal_enhancer[fetal_enhancer['Gene Type'] =='protein-coding']

In [ ]:
fetal_enhancer=fetal_enhancer.loc[fetal_unique_enhancer_id]
fetal_motif=fetal_motif.loc[fetal_unique_enhancer_id]

fetal_enhancer=fetal_enhancer.dropna()

In [ ]:
fetal_edge_df=fetal_network[['Motif Name New','Gene Name']]
fetal_edge_df.reset_index(inplace=True)
fetal_edge_df=fetal_edge_df.drop_duplicates(subset=['PositionID',"Motif Name New"],keep='first')
fetal_edge_df=fetal_edge_df.dropna()
fetal_gene_dict={}
for gene in list(set(fetal_edge_df["Motif Name New"])):
    fetal_gene_dict[gene]=list(fetal_edge_df[fetal_edge_df["Motif Name New"]==gene]["Gene Name"])